In [1]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from PIL import Image
from transformers import AutoModelForCausalLM
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
from deepseek_vl2.utils.io import load_pil_images
import torch
import gc
import traceback
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from torch.cuda.amp import autocast
import pickle


if __name__ == '__main__':
    cases_dir = '/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/'
    cache_dir = '/media/RLAB-Disk01/Large-Language-Models-Weights'
    model_id = "deepseek-ai/deepseek-vl2"
    offload_folder = '/media/RLAB-Disk01/Large-Language-Models-Weights'
    
    cases_to_process = []
    for case in os.listdir(cases_dir):
        case_dir = os.path.join(cases_dir, case)
        if os.path.isdir(case_dir):
            response_path = os.path.join(case_dir, 'deepseek-vl2.txt')
            if not os.path.exists(response_path):
                cases_to_process.append(case)
    print(f"Found {len(cases_to_process)} cases to process.")

    torch.set_default_tensor_type(torch.FloatTensor)
    processor = DeepseekVLV2Processor.from_pretrained(model_id, torch_dtype=torch.float16,
                                                     device_map="cpu", trust_remote_code=True, 
                                                     cache_dir=cache_dir, 
                                                     offload_folder=offload_folder)

    tokenizer = processor.tokenizer
    tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    tokenizer.add_special_tokens({'additional_special_tokens': ['<image>','<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>','<|User|>', '<|Assistant|>']})
    
    def build_conversation(case_dir, user_prompt,system_prompt):
        image_files = [f for f in os.listdir(case_dir) if f.lower().endswith('.png')]
        image_paths = [os.path.join(case_dir, f) for f in image_files]
        
        image_files= [image_files[-1]]
        image_paths = [image_paths[-1]]
        # Create image placeholders
        print(image_files)
        print(image_paths)
        image_tags = " ".join(["<image>"] * len(image_files))
        print(image_tags)
        
        return [
    
             {
                "role": "<|User|>",
                "content": f"{image_tags}\n<|ref|>{system_prompt}\n{user_prompt}<|/ref|>",
                "images": image_paths
            },
    
            {"role": "<|Assistant|>", "content": ""}
        ]
    
    failed_cases = []
    
    for case in cases_to_process:
        case_dir = os.path.join(cases_dir, case)
    
        clinical_path = os.path.join(case_dir, 'diagnostic_prompt.txt')
        if not os.path.exists(clinical_path):
                print(f"Missing clinical info for {case}")
                failed_cases.append(case)
                continue
                
        clinical_info = open(clinical_path).read()
            
    
        system_prompt = "Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.\n" +\
    "\n" +\
    "Use the following structure for the report:\n" +\
    "\n" +\
    "## Radiologist's Report\n" +\
    "\n" +\
    "### Patient Information:\n" +\
    "- *Age:* 65\n" +\
    "- *Sex:* Male\n" +\
    "- *Days from earliest imaging to surgery:* 1\n" +\
    "- *Histopathological Subtype:* Glioblastoma\n" +\
    "- *WHO Grade:* 4\n" +\
    "- *IDH Status:* Mutant\n" +\
    "- *Preoperative KPS:* 80\n" +\
    "- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21\n" +\
    "- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29\n" +\
    "- *Extent of Resection (EOR %):* 100.0\n" +\
    "- *EOR Type:* Gross Total Resection (GTR)\n" +\
    "- *Adjuvant Therapy:* Radiotherapy (RT) + Temozolomide (TMZ)\n" +\
    "- *Progression-Free Survival (PFS) Days:* 649\n" +\
    "- *Overall Survival (OS) Days:* 736\n" +\
    "\n" +\
    "#### Tumor Characteristics:\n" +\
    "\n" +\
    "#### Segmentation Analysis:\n" +\
    "\n" +\
    "#### Surgical Considerations:\n" +\
    "\n" +\
    "### Clinical Summary:\n" +\
    "\n" +\
    "### Recommendations:\n" +\
    "\n" +\
    "### Prognostic Considerations:\n" +\
    "\n" +\
    "### Follow-Up Plan:\n" +\
    "\n" +\
    "### Additional Notes*(if any)*:\n" +\
    "\n" +\
    "Ensure all findings from all of the images and clinical data provided. Please mention at the end of the report how many images were reviewed."
    
        user_prompt = f"You will be given batches of images, which are different sequences of MRI scans. \n" +\
    f"    The images are for patients who are likely to have a brain tumor. Each image will contain up to 10 slices for 5 different sequences and the segmentation masks for the tumor at the bottom row of the image. \n" +\
    f"    Additional clinical data about the patient is: \n" +\
    f"    {clinical_info}"
        # Prepare conversation
        conversation = build_conversation(case_dir, user_prompt,system_prompt)
        pil_images = load_pil_images(conversation)
        
        print(conversation)

        # Process inputs on CPU
        try:
            inputs = processor(
                conversations=conversation,
                images=pil_images,
                force_batchify=True,
                system_prompt=system_prompt
            )

            # Convert inputs to pickle file
            input_file_path = os.path.join(case_dir, 'inputs_deepseek.pkl')
            with open(input_file_path, 'wb') as f:
                pickle.dump(inputs, f, protocol=pickle.HIGHEST_PROTOCOL) #Added Higher protocol to pickle
            
            print(f"Inputs converted to pickle file on case {case}")

        except Exception as e:
             print(f"Error during input processing for case {case}: {e}")
             traceback.print_exc()
             failed_cases.append(case)
            
        
        del inputs, pil_images, conversation
        gc.collect()
        torch.cuda.empty_cache()
    
    # Save failed cases
    if failed_cases:
        with open(os.path.join(cases_dir, 'failed_deepseek-vl2.txt'), 'w') as f:
            f.write("\n".join(failed_cases))

/home/elboardy/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/elboardy/.local/lib/python3.9/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/elboardy/.local/lib/python3.9/site-packages/torchvision/image.so: undefined symbol: _ZN3c1017RegisterOperatorsD1Ev'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


Found 40 cases to process.


/home/elboardy/.local/lib/python3.9/site-packages/torch/__init__.py:1144: UserWarning: torch.set_default_tensor_type() is deprecated as of PyTorch 2.1, please use torch.set_default_dtype() and torch.set_default_device() as alternatives. (Triggered internally at ../torch/csrc/tensor/python_tensor.cpp:432.)
  _C._set_default_tensor_type(t)
/home/elboardy/.local/lib/python3.9/site-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


['RHUH-0019_batch_9.png']
['/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/RHUH-0019/RHUH-0019_batch_9.png']
<image>
[{'role': '<|User|>', 'content': "<image>\n<|ref|>Consider that you are a professional radiologist with several years of experience and you are now treating a patient. Write a fully detailed diagnosis report for this case, avoiding any potential hallucination and paying close attention to all of the batch images attached to this message.\n\nUse the following structure for the report:\n\n## Radiologist's Report\n\n### Patient Information:\n- *Age:* 65\n- *Sex:* Male\n- *Days from earliest imaging to surgery:* 1\n- *Histopathological Subtype:* Glioblastoma\n- *WHO Grade:* 4\n- *IDH Status:* Mutant\n- *Preoperative KPS:* 80\n- *Preoperative Contrast-Enhancing Tumor Volume (cm³):* 103.21\n- *Preoperative T2/FLAIR Abnormality (cm³):* 36.29\n- *Extent of Resection (EOR %):* 100.0\n- *EOR Type:* Gross Total Resection (GTR)\n- *Adjuvant The

In [13]:
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from transformers import AutoModelForCausalLM
from deepseek_vl2.models import DeepseekVLV2Processor, DeepseekVLV2ForCausalLM
import torch
import gc
import traceback
from accelerate import init_empty_weights, load_checkpoint_and_dispatch
from torch.cuda.amp import autocast
import pickle

if __name__ == '__main__':
    cases_dir = '/media/RLAB-Disk01/(final)merged_images_with_labels_order_and_folders_mask_normalized/'
    cache_dir = '/media/RLAB-Disk01/Large-Language-Models-Weights'
    model_id = "deepseek-ai/deepseek-vl2"
    offload_folder = '/media/RLAB-Disk01/Large-Language-Models-Weights'
    
    cases_to_process = []
    for case in os.listdir(cases_dir):
        case_dir = os.path.join(cases_dir, case)
        if os.path.isdir(case_dir):
            response_path = os.path.join(case_dir, 'deepseek-vl2.txt')
            if not os.path.exists(response_path):
                cases_to_process.append(case)
    print(f"Found {len(cases_to_process)} cases to process.")

    torch.set_default_tensor_type(torch.FloatTensor)
            # Loading the Model to GPU



    with init_empty_weights():
        model: DeepseekVLV2ForCausalLM = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.float16,
            trust_remote_code=True,
            use_safetensors=True,
            cache_dir=cache_dir,
            offload_folder=offload_folder,
        )
        
    # Load the model to the correct device
    model = load_checkpoint_and_dispatch(
        model,
        cache_dir+"/models--deepseek-ai--deepseek-vl2/snapshots/f363772d1c47f4239dd844015b4bd53beb87951b",
        device_map="auto",
        offload_folder=offload_folder,
        dtype=torch.float16,
    )
        
    processor = DeepseekVLV2Processor.from_pretrained(model_id, torch_dtype=torch.float16,
                                                     device_map="auto", trust_remote_code=True, 
                                                     cache_dir=cache_dir, 
                                                     offload_folder=offload_folder)
    

Found 40 cases to process.


Loading checkpoint shards: 100%|██████████| 8/8 [00:05<00:00,  1.59it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Add pad token = ['<｜▁pad▁｜>'] to the tokenizer
<｜▁pad▁｜>:2
Add image token = ['<image>'] to the tokenizer
<image>:128815
Add grounding-related tokens = ['<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>'] to the tokenizer with input_ids
<|ref|>:128816
<|/ref|>:128817
<|det|>:128818
<|/det|>:128819
<|grounding|>:128820
Add chat tokens = ['<|User|>', '<|Assistant|>'] to the tokenizer with input_ids
<|User|>:128821
<|Assistant|>:128822



In [15]:
    tokenizer = processor.tokenizer
    # tokenizer.add_special_tokens({'pad_token': '<|pad|>'})
    # tokenizer.add_special_tokens({'additional_special_tokens': ['<image>','<|ref|>', '<|/ref|>', '<|det|>', '<|/det|>', '<|grounding|>','<|User|>', '<|Assistant|>']})    
    model = model.eval()
    failed_cases = []
    
    for case in cases_to_process:
        case_dir = os.path.join(cases_dir, case)
        input_file_path = os.path.join(case_dir, 'inputs_deepseek.pkl')
        
        if not os.path.exists(input_file_path):
            print(f"Missing input pickle for {case}")
            failed_cases.append(case)
            continue

        try:
            with open(input_file_path, 'rb') as f:
                loaded_inputs = pickle.load(f)

            with torch.no_grad():
                # Prepare the necessary inputs for prepare_inputs_embeds
                prepared_inputs = {
                    "input_ids": loaded_inputs.input_ids.to(torch.long),  # Keep input_ids as Long
                    "attention_mask": loaded_inputs.attention_mask.to("cpu"),
                     "pixel_values": loaded_inputs.images.to(torch.float16) if hasattr(loaded_inputs,'images') else None,
                     "images_seq_mask": loaded_inputs.images_seq_mask.to("cpu") if hasattr(loaded_inputs,'images_seq_mask') else None

                   }
                inputs_embeds = model.prepare_inputs_embeds(**prepared_inputs)


                print(f"Messages prepared for case {case}")
                # Generate response
                outputs = model.language.generate(
                        inputs_embeds=inputs_embeds,
                        attention_mask=prepared_inputs["attention_mask"],
                        pad_token_id=tokenizer.pad_token_id,
                        bos_token_id=tokenizer.bos_token_id,
                        eos_token_id=tokenizer.eos_token_id,
                        max_new_tokens=4096,
                        temperature=0.7,
                        top_p=0.9,
                        do_sample=False,
                        use_cache=True,
                    )
            
                    
                # Decode and save response
            print(outputs.cpu().tolist())
            response = tokenizer.decode(outputs[0].cpu().tolist(), skip_special_tokens=True)
            
            print(f"Response generated for case {case}, length: {len(response)} characters")
            print(response)
            
            with open(os.path.join(case_dir, 'deepseek-vl2.txt'), 'w', encoding='utf-8') as f:
                f.write(response.strip())

            print("Response saved.",case)
        except Exception as e:
            print(f"Error during model prediction for case {case}: {e}")
            traceback.print_exc()
            failed_cases.append(case)

        del inputs_embeds, loaded_inputs,prepared_inputs
    #del model, processor
    gc.collect()
    torch.cuda.empty_cache()
    
    # Save failed cases
    if failed_cases:
        with open(os.path.join(cases_dir, 'failed_deepseek-vl2.txt'), 'w') as f:
            f.write("\n".join(failed_cases))

Messages prepared for case RHUH-0019
[[1]]
Response generated for case RHUH-0019, length: 0 characters

Response saved. RHUH-0019
Messages prepared for case RHUH-0001
[[361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 18, 201, 361, 

KeyboardInterrupt: 